# Перед первым использованием установите библиотеки

In [85]:
# !pip install pandas
# !pip install numpy
# !pip install pytest-warnings
# !pip install contextlib2
# !pip install psycopg2
# !pip install SQLAlchemy
# pip install beautifulsoup4
# pip install requests
# pip install more-itertools
# pip install fuzzywuzzy

## Навигация
- **[Формирование справочника стран](#Формирование-справочника-стран)**
- **[Сбор курса THB](#Сбор-курса-THB)**
- **[Сбор справочника измерений](#Сбор-справочника-измерений)**
- **[Словарь стран](#Словарь-стран)**
- **[Трансформация датафрейма](#Трансформация-датафрейма)**
- **[Загрузка данных в БД](#Загрузка-данных-в-БД)**
- **[Соборка кодов apk 24+](#Соборка-кодов-apk-24+)**

In [44]:
import pandas as pd
import numpy as np
import warnings
import psycopg2
from sqlalchemy import create_engine
import sqlite3
from datetime import datetime, timedelta, date
import os
import json
import warnings
warnings.simplefilter('ignore') 

In [45]:
from bs4 import BeautifulSoup
import requests

In [46]:
from itertools import product
from fuzzywuzzy import fuzz, process

In [47]:
# Инициализация подключений для работы с БД
engine = psycopg2.connect(user=os.getenv('USER'),
                          # пароль, который указали при установке PostgreSQL
                          password=os.getenv('PASSWORD'),
                          host=os.getenv('HOST'),
                          port=os.getenv('PORT'),
                          database=os.getenv('DATABASE'))

conn = create_engine('postgresql://{}:{}@{}:{}/{}'
                     .format(os.getenv('USER'), os.getenv('PASSWORD'), os.getenv('HOST'), os.getenv('PORT'), os.getenv('DATABASE')))


In [48]:
# Считываем конфиг названия таблиц и базы данных
with open('config_js_name_table_by_transform.json', 'r') as fl:
    name_table = json.load(fl)

In [49]:
# Считываем данные из БД sqlite
df_tmp_data = pd.read_sql(f"SELECT * FROM {name_table['name_tb_sqlite']}", con=sqlite3.connect(name_table['sqlite_db']))

# Формирование справочника стран 
*выполнить если необходимо актуализировать данные*

In [17]:
# Получаем разметку страницы для сборка названий стран из источника
reposns_country = requests.get('https://www.customs.go.th/statistic_report.php?lang=en&').text

In [18]:
# Получаем названия стран
need_soup_element = BeautifulSoup(reposns_country, 'lxml').find('select', id = 'country_code').find_all('option')

In [19]:
# Получаем нужные коды из БД
df_db_country = pd.read_sql(f"SELECT name_eng, code FROM {name_table['name_schema_reference']}.{name_table['name_tb_country']}  WHERE name_rus NOT LIKE '%недейств%'", 
                            con=engine)

In [20]:
# Получаем уже записанные коды из истоника для валидации дубликатов
df_sourse_thai = pd.read_sql(f"SELECT name_country_source FROM {name_table['name_schema_dl']}.{name_table['name_table_dl']} WHERE source_id = 5", 
                             con=engine)

In [21]:
# Словарь страна: код страны
dct_db_country = {k: v for k, v in zip(df_db_country.name_eng, df_db_country.code)}

In [22]:
# Список кодов из источника
l_in_source = [i.text.split(' - ')[-1] for i in need_soup_element if i.text.split(' - ')[-1] not in df_sourse_thai.name_country_source.tolist()]

In [110]:
# Скрипт сопоставления названия стран в источнике и БД
dct_for_df = {'name_eng': [], 'name_country_source': [], 'source_id': []}
for db, source in product(df_db_country.name_eng, l_in_source):
    
    if db == source.title() or fuzz.ratio(db, source.title()) > 65 and source != 'FRENCH SOUTHERN TERRITORIES':
        dct_for_df['name_eng'].append(db)
        dct_for_df['name_country_source'].append(source)
        dct_for_df['source_id'].append(5)
#         print(db, source)


In [ ]:
# Создаем датафрейм для загрузки в БД
df_to_postgre = pd.DataFrame(dct_for_df)

df_to_postgre['code'] = df_to_postgre.name_eng.apply(lambda x: dct_db_country[x])

In [ ]:
df_to_postgre

In [ ]:
# Загрузка данных
df_to_postgre.to_sql(name_table['name_table_dl'], con=conn, schema=name_table['name_schema_dl'], if_exists='append', index=False)

In [169]:
# Словари не сопоставленных стран
l_source = ['BOLIVIA', 'BOUVET ISLAND', 'CONGO, THE DEMOCRATIC REPUBLIC OF THE',  'CZECH REPUBLIC', 'EAST TIMOR', 'FRANCE', 'MONACO',
           'GUERNSEY', 'HONG KONG', 'IRAN (ISLAMIC REPUBLIC OF)', 'ISLE OF MAN', 'JERSEY', 'KOREA DPR', 'KOREA,REPUBLIC OF', 
           'LAO PDR', 'MACAU', 'MACEDONIA', 'MICRONESIA (FEDERATED STATES OF)', 'MOLDOVA REPUBLIC OF', 'SINT MAARTEN',
           'SVALBARD AND JAN MAYEN', 'SYRIAN ARAB REPUBLIC', 'TANZANIA UNITED REPUBLIC OF', 'TÜRKIYE', 'UNITED STATES', 'VATICAN CITY',
           'VIRGIN ISLANDS (BRITISH)', 'รหัสประเทศสำหรับเขตต่อเนื่องฯ', 'รหัสประเทศสำหรับเขตปลอดอากร', 'รหัสประเทศสำหรับเขตอุตสาหกรรมส่งออก']

l_db = ['Bolivia (Plurinational State of)', 'Ile Bouvet', 'Democratic Republic of the Congo', 'Czechia', 'Timor-Leste', 'France and Monaco', 'France and Monaco',
       'Bailiwick of Guernsey', 'China, Hong Kong SAR', 'Iran', 'Man Island', 'Bailiwick of Jersey', "Democratic People's Republic of Korea",
       'Rep. of Korea', "Lao People's Dem. Rep.", 'China, Macao Special Administrative Region', 'TFYR of Macedonia', 'Federated State of Micronesia',
       'Rep. of Moldova', 'Saint Martin (Franch part)', 'Norway, excluding Svalbard and Jan Mayen', 'Syria', 'United Rep. of Tanzania',
       'Turkey', 'USA, Puerto Rico and US Virgin Islands', 'Holy See (Vatican City State)', 'British Virgin Islands', 'Special Categories',
       'Free Zones', 'Bunkers']

In [183]:
l_source_otger = ['TAIWAN, PROVINCE OF CHINA', 'FRENCH SOUTHERN TERRITORIES', 'ZAIRE', 'DDD', 'FOR HIGH SEA ZONE', 'FRANCE, METROPOLITAN',
             'LIBYAN ARAB JAMAHIRIYA', 'OTHER COUNTRY', 'PALESTINIAN TERRITORY, OCCUPIED']

l_db_otger = ['Other Asia, not elsewhere specified', 'French South Antarctic Territories', 'Democratic Republic of the Congo',
                 'Areas, not elsewhere specified', 'Areas, not elsewhere specified', 'Areas, not elsewhere specified',
                 'Areas, not elsewhere specified', 'Areas, not elsewhere specified', 'Areas, not elsewhere specified']

In [184]:
dct_other_val = {'name_eng': l_db_otger, 'name_country_source': l_source_otger}

In [185]:
df_for_postgre_2 = pd.DataFrame(dct_other_val)
df_for_postgre_2['source_id'] = 5
df_for_postgre_2['code'] = df_for_postgre_2.name_eng.apply(lambda x: dct_db_country[x])

In [186]:
df_for_postgre_2.to_sql(name_table['name_table_dl'], con=conn, schema=name_table['name_schema_dl'], if_exists='append', index=False)

9

# Сбор курса THB

In [143]:
# https://ru.investing.com/currencies/usd-thb-historical-data
# ссылка для скачивания файла

In [50]:
df_bth = pd.read_csv('Прошлые данные - USD_THB.csv')

In [51]:
df_bth['price'] = df_bth['Цена'].apply(lambda x: float(x.replace(',', '.')))

In [52]:
df_bth['period'] = df_bth['Дата'].apply(lambda x: '-'.join(x.split('.')[::-1]))

In [53]:
dct_currency = {k: v for k, v in zip(df_bth.period, df_bth.price)}

In [54]:
dct_currency

{'2023-10-01': 36.414,
 '2023-09-01': 36.51,
 '2023-08-01': 34.98,
 '2023-07-01': 34.21,
 '2023-06-01': 35.27,
 '2023-05-01': 34.64,
 '2023-04-01': 34.12,
 '2023-03-01': 34.16,
 '2023-02-01': 35.22,
 '2023-01-01': 32.84}

# Сбор справочника измерений

In [55]:
# Провермяем все ли единици измерения учтены
# Если нет, то добавить новые в словарь dct_deizm
# Значение является измерением, если имеет вид (название_измерения)
df_tmp_data['desc'] = df_tmp_data.description.apply(lambda x: x.split()[-1])
df_tmp_data.desc.unique()

array(['(C62)', '(KGM)', 'seeds', 'spinosum', 'fractions', 'wax)',
       'beans', '48', 'Other', 'sale', '(LTR)', 'frozen', 'maize',
       '(RBD)'], dtype=object)

In [56]:
dct_deizm = {
    
    'KGM': 8,
    'C62': 5,
    'LTR': 7,
    'OTHER': 1
}

# Словарь стран

In [57]:
# Создаем словарь стран, для валидации кодов партнеров
df_country_db = pd.read_sql(f"SELECT code, name_country_source FROM {name_table['name_schema_dl']}.{name_table['name_table_dl']} WHERE source_id = 5",
                           con=engine)
dct_county = {k.strip(): v for k, v in zip(df_country_db.name_country_source, df_country_db.code)}

# Трансформация датафрейма

In [58]:
# Меняем название стран партнеров на их коды
df_tmp_data['partner_code'] = df_tmp_data.partner_code.apply(lambda x: dct_county[x])

In [59]:
# Достаем нужные названия измерений
df_tmp_data['desc'] = df_tmp_data.description.apply(lambda x: x.split()[-1].replace('(', '').replace(')', '') 
                                                    if x.split()[-1].replace('(', '').replace(')', '') in dct_deizm
                                                    else 'OTHER')
# Корректируем коды qty_unit_code
df_tmp_data['qty_unit_code'] = df_tmp_data.desc.apply(lambda x: dct_deizm[x])

In [60]:
# df_tmp_data['qty'] = df_tmp_data.apply(lambda x: x['netweight'] if x['qty_unit_code'] != 8 else 0, axis=1)
df_tmp_data['qty'] = df_tmp_data['netweight']
df_tmp_data['netweight'] = df_tmp_data.apply(lambda x: x['netweight'] if x['qty_unit_code'] == 8 else 0, axis=1)

In [61]:
# Приводим trade_value к USD 
df_tmp_data['trade_value'] = df_tmp_data.apply(lambda x: x['trade_value'] / dct_currency[x['period']], axis=1)

In [62]:
# Проверка на дубли
df_tmp_data.groupby(['period', 'trade_flow_code', 'partner_code', 'commodity_code']).agg({'qty': 'count'}).query('qty > 1')

qty
period     trade_flow_code partner_code commodity_code     
2023-01-01 1               251          13023990000       2
           2               899          16041419000       2
2023-02-01 1               251          13021990090       2
2023-03-01 1               251          13023990000       2
2023-04-01 1               251          13023990000       2
2023-05-01 2               899          16041419000       2
2023-06-01 1               251          13023990000       2

In [63]:
# Смотрим на дублируемые данные
df_tmp_data.query("period == '2023-01-01' and partner_code == 251 and  commodity_code == '13023990000'")

,classification,year,period,aggregate_level,trade_flow_code,region_code,reporter_code,partner_code,customs_proc_code,commodity_code,qty_unit_code,qty,netweight,trade_value,flag,plus,load_mark,update_date,description,desc
19609,HS,2023,2023-01-01,11,1,NNNNN,764,251,C00,13023990000,8,1347.0,1347.0,97784.378806,0,0,1,2023-10-11,Other ** Other (KGM),KGM
19616,HS,2023,2023-01-01,11,1,NNNNN,764,251,C00,13023990000,8,5.0,5.0,289.890378,0,0,1,2023-10-11,Other ** Other (KGM),KGM


In [64]:
# Так как в данных есть страны, которые у нас относятся к одному коду, просуммируем их 
# Пример Франция и Монако
df_tmp_data = df_tmp_data.groupby(['classification', 'year', 'period', 'aggregate_level', 'trade_flow_code', 'region_code', 'reporter_code',
                     'partner_code', 'customs_proc_code', 'commodity_code', 'qty_unit_code', 'flag', 'plus', 'load_mark', 'update_date'], as_index=False) \
                .agg({'qty': 'sum', 'netweight': 'sum', 'trade_value': 'sum'})

In [65]:
# Создаем колонки cif и fob
df_tmp_data['cif_trade_value'] = df_tmp_data.apply(lambda x: x['trade_value'] if x['trade_flow_code'] == 1 else None, axis=1)
df_tmp_data['fob_trade_value'] = df_tmp_data.apply(lambda x: x['trade_value'] if x['trade_flow_code'] == 2 else None, axis=1)

In [66]:
# # Удаляем не нужные колонки
# df_tmp_data.drop(columns=['description', 'desc'], inplace=True)

In [67]:
df_tmp_data.head(2)

,classification,year,period,aggregate_level,trade_flow_code,region_code,reporter_code,partner_code,customs_proc_code,commodity_code,qty_unit_code,flag,plus,load_mark,update_date,qty,netweight,trade_value,cif_trade_value,fob_trade_value
0,HS,2023,2023-01-01,11,1,NNNNN,764,4,C00,08062000000,8,0,0,1,2023-10-11,20000.0,20000.0,33367.844093,33367.844093,NaN
1,HS,2023,2023-01-01,11,1,NNNNN,764,32,C00,02013000001,8,0,0,1,2023-10-10,3156.0,3156.0,14771.833130,14771.833130,NaN


In [68]:
df_tmp_data.shape

(167435, 20)

# Загрузка данных в БД

In [69]:
# Очищаем данные из БД
with engine.cursor() as cur:
    cur.execute(f'DELETE FROM {name_table["name_schema_main"]}.{name_table["name_table_main"]} WHERE reporter_code = 764')
    engine.commit()

In [72]:
df_count_value = pd.read_sql(f'SELECT COUNT(*) AS count_value FROM {name_table["name_schema_main"]}.{name_table["name_table_main"]} WHERE reporter_code = 764', 
                             con=engine)
сheck_zero = df_count_value.count_value[0]

In [71]:
if сheck_zero == 0:
    print(f'записей в таблице: {сheck_zero}, начинаем загрузку')
    df_tmp_data.to_sql(name_table["name_table_main"], con=conn, schema=name_table["name_schema_main"],  
                                                                          if_exists='append', index=False)
else:
    print(f'записей в таблице: {сheck_zero}, произведите удаление лишних элементов')

записей в таблице: 0, начинаем загрузку


In [73]:
сheck_zero

167435

# Соборка кодов apk 24+

*Выполнить если необходимо обновить коды*

In [164]:
# Получаем все коды
df_cd = pd.read_sql(f"SELECT code FROM {name_table['name_schema_reference']}.{name_table['name_tb_tnved_code']} WHERE type = 10 AND prod_type = 'apk'", con=engine)

In [165]:
# Валидируем коды
df_cd['bool_col'] = df_cd.code.apply(lambda x: int(x[:2]) > 24 if x.isdigit() else False ) 

In [166]:
df_cd = df_cd[df_cd.bool_col]

In [167]:
# Отбираем нужное
df_cd['need_val'] = df_cd.code.apply(lambda x: x[:4])

In [168]:
len(df_cd.need_val.unique())

40

In [293]:
# Сохраняем в файл
with open('25_apk.txt', 'w') as fl:
    fl.write(' '.join(df_cd.need_val.unique().tolist()))

### [⬅ Навигация](#Навигация)